In [1]:

import os
import sys
from os import path
import pause

import threading
from threading import Timer, Thread
import schedule

import pandas as pd
import numpy as np

from datetime import datetime, timedelta, time
import time as ot

import requests
from public_data_config import apisdata

from get_holiday import _get_holiday
from make_stock_list import _get_stock_list
from make_cci_signal import _make_cci_signal
from collect_signal import _collect_cci_signal

from slack_message import _post_message
from slack_sdk_message_post import _post_message_with_slack_sdk

In [2]:
class BaseService:
    def __init__(self):
        self.myToken = apisdata["slack"]["token"]
        self.channel_name = apisdata["slack"]["channel_name"]
        self.channel_id = apisdata["slack"]["channel_id"]

        # 휴장일 데이터 가져오기
        self.mykey = apisdata["holiday"]["decoding_key"]
        self.request_url = apisdata["holiday"]["request_url"]
        self.year = "2022"
        self.market = "kospi"
        self.market_stock_dict = {
            self.market : None,
            }
        self.market_dict = None
        self.config_ndays = 20
        self.financial_engineering_days = 252
        self.start_date = (datetime.now() - timedelta(days=self.financial_engineering_days)).date().strftime("%Y-%m-%d")
        self.ko_holiday_list =  ['2022-01_01','2022-01_31','2022-02_01','2022-02_02','2022-03_01','2022-03_09','2022-05_05','2022-05_08','2022-06_01','2022-06_06','2022-08_15','2022-09_09','2022-09_10','2022-09_11','2022-09_12','2022-10_03','2022-10_09','2022-10_10','2022-12_25']
        self.target_rtn = 3.0
        self.target_days = 5
        self.max_trade_stock_count = 10

In [5]:
market_stock_dict = _get_stock_list(BaseService())
len(market_stock_dict[BaseService().market])

Do function : _get_stock_list


200

In [10]:
import yfinance as yf
total_df = pd.DataFrame()
for symbol in list(market_stock_dict["kospi"].keys()):
    df = yf.download(symbol+".KS", start=BaseService().start_date, auto_adjust = True, action=True, threads=False).reset_index()
    # y_df["Symbol"] = symbol
    # signal_dict, df =_make_cci_signal(BaseService(), y_df)
    # total_df = total_df.append(df)
    break

df.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume
0,2021-08-18,169050.0,170030.0,159740.0,160230.0,69302
1,2021-08-19,160230.0,160720.0,157290.0,158760.0,20208
2,2021-08-20,159250.0,160230.0,155820.0,155820.0,24596
3,2021-08-23,157290.0,166600.0,157290.0,164150.0,48616
4,2021-08-24,165130.0,169540.0,165130.0,169540.0,20517


In [13]:
df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["sma"] = df["TP"].rolling(20).mean()
df["mad"] = df["TP"].rolling(20).apply(lambda x: pd.Series(x).mad())
df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])
df["pre1day_CCI"] = df["CCI"].shift(1)

In [21]:
df.head(40)

,Date,Open,High,Low,Close,Volume,TP,sma,mad,CCI,pre1day_CCI,buy_price,target_price
0,2021-08-18,169050.0,170030.0,159740.0,160230.0,69302,163333.333333,NaN,NaN,NaN,NaN,None,None
1,2021-08-19,160230.0,160720.0,157290.0,158760.0,20208,158923.333333,NaN,NaN,NaN,NaN,None,None
2,2021-08-20,159250.0,160230.0,155820.0,155820.0,24596,157290.000000,NaN,NaN,NaN,NaN,None,None
3,2021-08-23,157290.0,166600.0,157290.0,164150.0,48616,162680.000000,NaN,NaN,NaN,NaN,None,None
4,2021-08-24,165130.0,169540.0,165130.0,169540.0,20517,168070.000000,NaN,NaN,NaN,NaN,None,None
5,2021-08-25,166600.0,172480.0,165620.0,171990.0,31915,170030.000000,NaN,NaN,NaN,NaN,None,None
6,2021-08-26,169540.0,174930.0,169050.0,174440.0,26850,172806.666667,NaN,NaN,NaN,NaN,None,None
7,2021-08-27,173950.0,174440.0,171010.0,173460.0,20551,172970.000000,NaN,NaN,NaN,NaN,None,None
8,2021-08-30,172970.0,177380.0,172970.0,177380.0,27053,175910.000000,NaN,NaN,NaN,NaN,None,None
9,2021-08-31,176890.0,179340.0,170520.0,174930.0,40176,174930.000000,NaN,NaN,NaN,NaN,None,None


In [20]:
df["buy_price"] = np.where((df["pre1day_CCI"] < -100) & (df["CCI"]>=-100), df["Close"], None)
df["target_price"] = np.where(~df["buy_price"].isna(), df["buy_price"] + df["buy_price"] * 0.012, None )

In [ ]:
df[]

In [12]:
df["buy_price"] = None
for i, x in df.iterrows():
    df.loc[i, "buy_price"] = 
    break

nan


In [ ]:
print(round(total_df[total_df["trade"]=="Buy"].groupby(["Date"])["Symbol"].nunique().reset_index(name="buy_count")["buy_count"].mean(), 2))
print(round(total_df[total_df["trade"]=="Sell"].groupby(["Date"])["Symbol"].nunique().reset_index(name="sell_count")["sell_count"].mean(), 2))

In [ ]:
total_df["Date"].iloc[-1]

In [ ]:
total_df[total_df["trade"]=="Buy"].sort_values("Date")

In [ ]:
buy_df = total_df[total_df["trade"] == "Buy"].sort_values("Date").reset_index(drop=True)
buy_df[buy_df["Date"]==buy_df["Date"].iloc[-1]]

In [ ]:
print(total_df[(total_df["trade"]=="Buy") & (total_df["Date"] == total_df["Date"].iloc[-1])]["Symbol"].nunique())
total_df[(total_df["trade"]=="Sell") & (total_df["Date"] == total_df["Date"].iloc[-1])]["Symbol"].nunique()

In [ ]:
print(round(total_df[total_df["trade"]=="Buy"].iloc[-1]["Symbol"].nunique(), 2))
print(round(total_df[total_df["trade"]=="Sell"].iloc[-1]["Symbol"], 2))

In [ ]:
df, buy_stock_list, sell_stock_list = _collect_cci_signal(BaseService(), market_stock_dict)
df.head()

In [ ]:
buy_df = df[df["name"].isin(buy_stock_list)]
sell_df = df[df["name"].isin(sell_stock_list)]
stock_info_list = []

for i, x in buy_df.iterrows():
    candidate_dict = {
        "type" : "section",
        "text" : {
            "type" : "mrkdwn",
            "text" : f'매매신호 : {x["trade_signal"]}\n\
이름 : {x["name"]}\n\
symbol : {x["symbol"]}\n\
매매신호가격 : {x["signal_price"]}\n\
CCI수익률-구매후보유수익률 : {x["cci_buy_and_hold_diff_rtn"]}\n\
{BaseService().start_date}CCI수익률 : {x["cci_rtn"]}\n\
{BaseService().start_date}구매후보유수익률 : {x["buy_and_hold_rtn"]}\n\
{BaseService().target_days}일 이내 평균최고수익률 : {x["max_rtn"]}\n\
{BaseService().target_days}일 이내 구매 후 평균최고수익일수 : {x["max_rtn_day"]}\n\
CCI전략수행시 보유주식수 : {x["remain_holding_shares"]}\n\
CCI전략수행시 보유주식 평균 구매가격: {x["holding_shares_buy_price"]}'
}
}
    stock_info_list.append(candidate_dict)

for i, x in sell_df.iterrows():
    candidate_dict = {
        "type" : "section",
        "text" : {
            "type" : "mrkdwn",
            "text" : f'매매신호 : {x["trade_signal"]}\n\
이름 : {x["name"]}\n\
symbol : {x["symbol"]}\n\
매매신호가격 : {x["signal_price"]}\n\
CCI수익률-구매후보유수익률 : {x["cci_buy_and_hold_diff_rtn"]}\n\
{BaseService().start_date}CCI수익률 : {x["cci_rtn"]}\n\
{BaseService().start_date}구매후보유수익률 : {x["buy_and_hold_rtn"]}\n\
{BaseService().target_days}일 이내 평균최고수익률 : {x["max_rtn"]}\n\
{BaseService().target_days}일 이내 구매 후 평균최고수익일수 : {x["max_rtn_day"]}\n\
CCI전략수행시 보유주식수 : {x["remain_holding_shares"]}\n\
CCI전략수행시 보유주식 평균 구매가격: {x["holding_shares_buy_price"]}'
}
}
    stock_info_list.append(candidate_dict)
print(stock_info_list)

In [ ]:
_post_message_with_slack_sdk(BaseService(), blocks=stock_info_list)

In [ ]:
buy_stock_list = df[df["trade_signal"]=="Buy"].sort_values(["cci_buy_and_hold_diff_rtn", f"within_{BaseService().target_days}days_expected_rtn"], ascending=False)["name"].tolist()[:BaseService().max_trade_stock_count]
buy_stock_list

In [ ]:
sell_stock_list = df[df["trade_signal"]=="Sell"].sort_values(["cci_buy_and_hold_diff_rtn", f"within_{BaseService().target_days}days_expected_rtn"])["name"].tolist()[:BaseService().max_trade_stock_count]
sell_stock_list

In [ ]:
df[df["name"].isin(buy_stock_list)]

In [ ]:
t_list = range(1, 6)
t_list

In [ ]:
df[df["trade_signal"]=="Sell"].sort_values(["cci_buy_and_hold_diff_rtn", "within_7days_expected_rtn"])

In [ ]:
df[df["trade_signal"]=="Buy"].sort_values(["cci_buy_and_hold_diff_rtn", "within_7days_expected_rtn"], ascending=False)

In [ ]:
r_df[["after_1day_price", "Close"]]

In [ ]:
class BaseService:
    def __init__(self):
        self.myToken = apisdata["slack"]["token"]
        self.channel = apisdata["slack"]["channel"]

        # 휴장일 데이터 가져오기
        self.mykey = apisdata["holiday"]["decoding_key"]
        self.request_url = apisdata["holiday"]["request_url"]
        self.year = "2022"
        self.market = "kospi"
        self.market_stock_dict = {
            self.market : None,
            }
        self.market_dict = None
        self.config_ndays = 20
        self.start_date = (datetime.now() - timedelta(days=40)).date().strftime("%Y-%m-%d")
        self.ko_holiday_list =  ['2022-01_01','2022-01_31','2022-02_01','2022-02_02','2022-03_01','2022-03_09','2022-05_05','2022-05_08','2022-06_01','2022-06_06','2022-08_15','2022-09_09','2022-09_10','2022-09_11','2022-09_12','2022-10_03','2022-10_09','2022-10_10','2022-12_25']

    def scheduler(self):
        today_date = datetime.now().date().strftime("%Y-%m-%d")
        try:
            holiday_list = _get_holiday(self)
        except Exception as e:
            print(e)
            holiday_list = []

        if today_date not in holiday_list:
            try:
                self._get_stock_list()

                self._post_signal_to_slack()
            except Exception as e:
                self._post_message(e)
                pass

        else:
            try:
                self._get_stock_list()
                self._post_signal_to_slack()
            except Exception as e:
                self._post_message(e)
                pass
    
    def _get_stock_list(self):
        stock_list = []
        if self.market == "kospi":
            for i in range(1, 21):
                page = i
                url = 'https://finance.naver.com/sise/entryJongmok.nhn?&page={page}'.format(page = page)
                source = urlopen(url).read()
                source = BeautifulSoup(source,'lxml')
                source = source.find_all('a',target = '_parent')
                for j in range(len(source)):
                    name = source[j].text
                    stock_list.append(name)

            kospi_df = fdr.StockListing(self.market)
            self.market_dict = kospi_df[kospi_df["Name"].isin(stock_list)].set_index("Symbol")["Name"].to_dict()
            self.market_stock_dict[self.market] = self.market_dict

        elif self.market == "sp500":
            self.market_dict = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].set_index("Symbol")["Security"].to_dict()
            self.market_stock_dict[self.market] = self.market_dict


    def _post_message(self, text):
        response = requests.post("https://slack.com/api/chat.postMessage",
            headers={"Authorization": "Bearer "+self.myToken},
            data={"channel": self.channel,"text": text}
        )
        if response.ok:
            print("Success. Send message with slack.")
        else:
            print("Fail. Send message with slack.")
            pass

    def _make_signal(self, df):
        try:
            df = df.reset_index()
            df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
            df["sma"] = df["TP"].rolling(self.config_ndays).mean()
            df["mad"] = df["TP"].rolling(self.config_ndays).apply(lambda x: pd.Series(x).mad())
            df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])
            
            df["pre1day_cci"] = df["CCI"].shift(1)
            df["trade"] = None
            df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
            df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])
            
            df["total_buy_price"] = 0
            df["shares"] = 0
            df["buy_price"] = 0
            df["sell_price"] = 0
            df["revenue"] = 0
            df["rate"] = 0

            buy_price_list = []
            for i, x in df.iterrows():
                if x["trade"] == "buy":
                    buy_price_list.append(x["Close"])
                    df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                    df.loc[i, "shares"] = len(buy_price_list)
                    df.loc[i, "buy_price"] = np.mean(buy_price_list)
                elif x["trade"] == None:
                    if i > 0:
                        df.loc[i, "shares"] = df.loc[i-1, "shares"]
                        df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
                        df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
                elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
                    first_price = buy_price_list.pop(0)
                    df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
                    df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
                    df.loc[i, "sell_price"] = df.loc[i, "Close"]
                    df.loc[i, "shares"] = len(buy_price_list)
                    if len(buy_price_list) > 0:
                        df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                        df.loc[i, "buy_price"] = np.mean(buy_price_list)
                    else:
                        df.loc[i, "total_buy_price"] = 0
                        df.loc[i, "buy_price"] = 0
            
            r_dict = {
                "symbol" : df.iloc[-1]["Symbol"],
                "date" : df.iloc[-1]["Date"],
                "trade_signal" : df.iloc[-1]["trade"],
                "price" : df.iloc[-1]["Close"]
            }
            return r_dict
            
        except Exception as e:
            # self._post_message(e)
            print(e)
            pass

    def _post_signal_to_slack(self):
        df = pd.DataFrame()
        buy_list = []
        sell_list = []
        idata= []

        if self.market == "kospi":
            for symbol in self.market_stock_dict[self.market]:
                try:
                    df = fdr.DataReader(symbol, start=self.start_date)
                    df["Symbol"] = symbol

                    r_dict = self._make_signal(df)

                    if r_dict["trade_signal"] == "buy":
                        buy_list.append(self.market_dict.get(r_dict["symbol"]))
                        print(r_dict)
                        idata.append(r_dict)
                    elif r_dict["trade_signal"] == "sell":
                        sell_list.append(self.market_dict.get(r_dict["symbol"]))
                        print(r_dict)
                        idata.append(r_dict)
                except Exception as e:
                    # self._post_message(e)
                    print(e)
                    pass

        elif self.market == "sp500":
            for symbol in self.market_stock_dict[self.market]:
                try:
                    df = yf.download(symbol, start=self.start_date, show_errors=False)
                    df["Symbol"] = symbol
                    
                    r_dict = self._make_signal(df)

                    if r_dict["trade_signal"] == "buy":

                        buy_list.append(r_dict["symbol"])
                        print(r_dict)
                        idata.append(r_dict)
                    elif r_dict["trade_signal"] == "sell":
                        sell_list.append(r_dict["symbol"])
                        print(r_dict)
                        idata.append(r_dict)

                except Exception as e:
                    # self._post_message(e)
                    print(e)
                    pass
            
        # 구매 데이터
        if len(buy_list) == 0:
            self._post_message("Today not exists buy stocks")
        if len(buy_list) > 0:
            self._post_message("Buy_stocks : %s"%(buy_list))

        # 판매 데이터
        if len(sell_list) == 0:
            self._post_message("Today not exists sell stocks")
        if len(sell_list) > 0:
            self._post_message("Sell_stocks : %s"%(sell_list))
        
        try:
            df = pd.DataFrame.from_dict(idata)
        except Exception as e:
            self._post_message(e)
            pass

        return df
        
    def _get_priority_house_jungso(self):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        url = "https://www.smes.go.kr/sanhakin/websquare/wq_main.do"
        driver.get(url)
        driver.implicitly_wait(10)
        driver.find_element(By.XPATH, '//*[@id="genTopMenu_2_liTopMenu"]').click()
        driver.implicitly_wait(1)
        driver.find_element(By.XPATH, '//*[@id="genLeftMenu_3_leftMenuGrp"]').click()
        driver.implicitly_wait(1)

        idata = []
        # pagelist1_page_2
        search_date = datetime.today().date()
        str_search_date = datetime.strftime(search_date, "%Y-%m-%d")
        for page in range(1, 3):
            # print(page)
            driver.find_element(By.XPATH, '//*[@id="pagelist1_page_%s"]'%page).click()

            driver.implicitly_wait(1)
            for table_order in range (0, 10):
                title = driver.find_element_by_css_selector('#gridView1_cell_%s_0'%table_order).text
                str_start_date = driver.find_element_by_css_selector('#gridView1_cell_%s_3'%table_order).text.split(' ')[0]
                start_date = datetime.strptime(str_start_date, "%Y-%m-%d").date()
                if search_date <= start_date:
                    idict = {
                        "search_date" : str_search_date,
                        "title" : title,
                        "start_date" : str_start_date
                        }
                    idata.append(idict)
            if search_date > datetime.strptime(driver.find_element_by_css_selector('#gridView1_cell_9_3').text.split(' ')[0], "%Y-%m-%d").date():
                print(datetime.strptime(driver.find_element_by_css_selector('#gridView1_cell_9_3').text.split(' ')[0], "%Y-%m-%d").date())
                break
        driver.quit()
        if len(idata) > 0:
            text = "유효한 새로운 중소기업 장기근속자 주택 특별공급 없음"
            _post_message(self, text)
        else:
            text = idata
            _post_message(self, text)
        return idata

    def work(self):
        if __name__ == "__main__":
            self._get_stock_list()
        return self.market_stock_dict


In [ ]:
market_stock_dict = BaseService().work()

In [ ]:
ilist = []
for stock in market_stock_dict["kospi"]:
    symbol = stock+".KS"
    ilist.append(yf.Ticker(symbol).info)
market_info_df = pd.DataFrame(ilist)
market_info_df.head()

In [ ]:
kospi_df = pd.DataFrame(ilist)
kospi_df.head()

In [ ]:
kospi_df['sharesOutstanding']

In [ ]:
kospi_df[kospi_df["symbol"]=='282330.KS']

In [ ]:
financial_engineering_days = 252
end_date = datetime.today().date()
str_end_date = datetime.strftime(end_date, "%Y-%m-%d")
start_date = (end_date - timedelta(days=financial_engineering_days))
str_start_date = datetime.strftime(start_date, "%Y-%m-%d")
print(str_start_date)
print(str_end_date)


In [ ]:
import yfinance as yf

In [ ]:
yf.download("282330.KS", auto_adjust=True).reset_index()

In [ ]:
financial_engineering_days = 252
end_date = datetime.today().date()
str_end_date = datetime.strftime(end_date, "%Y-%m-%d")
start_date = (end_date - timedelta(days=financial_engineering_days))
str_start_date = datetime.strftime(start_date, "%Y-%m-%d")
idata = []
for stock in market_stock_dict["kospi"]:
    df = pd.DataFrame()
    symbol = stock+".KS"
    print(symbol)
    datetime.today().date()
    try:
        df = yf.Ticker(symbol).history(interval="1d", start = str_start_date, end = str_end_date, auto_adjust=True).reset_index()
        df = df[df["Volume"]>0].reset_index(drop=True)
        config_ndays = 20
        
        # 수익률 계산하기
        df["daily_rtn"] = df["Close"].pct_change()  # 퍼센트 변화율
        df["st_rtn"] = (1 + df["daily_rtn"]).cumprod()  # 누적곱 계산함수 return cumulative product over a DataFrame or Series axis.
        gagr = (df.iloc[-1]['st_rtn'] ** (financial_engineering_days / len(df.index)) - 1) * 100

        df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
        df["sma"] = df["TP"].rolling(config_ndays).mean()
        df["mad"] = df["TP"].rolling(config_ndays).apply(lambda x: pd.Series(x).mad())
        df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])

        df["pre1day_cci"] = df["CCI"].shift(1)
        df["trade"] = None
        df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
        df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])

        df["total_buy_price"] = 0
        df["shares"] = 0
        df["buy_price"] = 0
        df["sell_price"] = 0
        df["revenue"] = 0
        df["rate"] = 0

        buy_price_list = []
        for i, x in df.iterrows():
            if x["trade"] == "buy":
                buy_price_list.append(x["Close"])
                df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                df.loc[i, "shares"] = len(buy_price_list)
                df.loc[i, "buy_price"] = np.mean(buy_price_list)
            elif x["trade"] == None:
                if i > 0:
                    df.loc[i, "shares"] = df.loc[i-1, "shares"]
                    df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
                    df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
            elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
                first_price = buy_price_list.pop(0)
                df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
                df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
                df.loc[i, "sell_price"] = df.loc[i, "Close"]
                df.loc[i, "shares"] = len(buy_price_list)
                if len(buy_price_list) > 0:
                    df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                    df.loc[i, "buy_price"] = np.mean(buy_price_list)
                else:
                    df.loc[i, "total_buy_price"] = 0
                    df.loc[i, "buy_price"] = 0

        r_dict = {
            "symbol" : symbol,
            "date" : datetime.strftime(df.iloc[-1]["Date"], "%Y-%m-%d"),
            "trade_signal" : df.iloc[-1]["trade"],
            "price" : df.iloc[-1]["Close"],
            "remain_shares" : df.iloc[-1]["shares"],
            "holding_shares_buy_price" : df.iloc[-1]["buy_price"],
            "cci_rtn" : df[df["trade"] == "sell"].iloc[-1]["rate"],
            "buy_and_hold_rtn" : gagr
        }
        idata.append(r_dict)
    except Exception as e:
        print(e)
        pass
    print(r_dict)

In [ ]:
t_df = pd.DataFrame(idata)
t_df[t_df["trade_signal"]=="buy"]

In [ ]:
print(idata)

In [ ]:
df[df["trade"]]

In [ ]:
df[df["rate"]>0]

In [ ]:
df.info()

In [ ]:
df[df["sma"]>0]

In [ ]:
df["sma"] = df["TP"].rolling(config_ndays).mean()
df["sma"]

In [ ]:
df[df["TP"]>0]

In [ ]:
ilist = []
for stock in market_stock_dict["kospi"]:
    symbol = stock+".ks"
    ilist.append(yf.Ticker(symbol).info)

In [ ]:
for stock in market_stock_dict["kospi"]:
    

In [ ]:
df = yf.Ticker('282330.ks').history(interval="1d", start="2021-01-01", end="2022-04-11").reset_index()
df["Symbol"] = "282330"

In [ ]:

config_ndays = 20
df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["sma"] = df["TP"].rolling(config_ndays).mean()
df["mad"] = df["TP"].rolling(config_ndays).apply(lambda x: pd.Series(x).mad())
df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])

df["pre1day_cci"] = df["CCI"].shift(1)
df["trade"] = None
df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])

df["total_buy_price"] = 0
df["shares"] = 0
df["buy_price"] = 0
df["sell_price"] = 0
df["revenue"] = 0
df["rate"] = 0

buy_price_list = []
for i, x in df.iterrows():
    if x["trade"] == "buy":
        buy_price_list.append(x["Close"])
        df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
        df.loc[i, "shares"] = len(buy_price_list)
        df.loc[i, "buy_price"] = np.mean(buy_price_list)
    elif x["trade"] == None:
        if i > 0:
            df.loc[i, "shares"] = df.loc[i-1, "shares"]
            df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
            df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
    elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
        first_price = buy_price_list.pop(0)
        df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
        df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
        df.loc[i, "sell_price"] = df.loc[i, "Close"]
        df.loc[i, "shares"] = len(buy_price_list)
        if len(buy_price_list) > 0:
            df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
            df.loc[i, "buy_price"] = np.mean(buy_price_list)
        else:
            df.loc[i, "total_buy_price"] = 0
            df.loc[i, "buy_price"] = 0

r_dict = {
    "symbol" : df.iloc[-1]["Symbol"],
    "date" : df.iloc[-1]["Date"],
    "trade_signal" : df.iloc[-1]["trade"],
    "price" : df.iloc[-1]["Close"]
}

In [ ]:
df.columns

In [ ]:
df[~df["trade"].isnull()][["Symbol", "Date", "trade", "shares", "buy_price", "sell_price", "total_buy_price", "revenue", "rate"]]

In [ ]:
df['revenue'].sum() / df[df["trade"]=="buy"]["buy_price"].sum()

In [ ]:
def returns(book):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        # long 진입
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, 'adjclose']
            print('진입일 : ', i, 'long 진입 가격 : ', buy)
        # long 청산
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':
            sell = book.loc[i, 'adjclose']
            rtn = (sell - buy) / buy + 1 #  손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ', i, 'long 진입 가격 : ', buy, 'long 청산 가격 : ', sell, '| return : ', round(rtn, 4))
        if book.loc[i, 'trade'] == '':  # 제로 포지션
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn =  acc_rtn * rtn  # 누적 수익률 계산
        book.loc[i, 'acc_return']  = acc_rtn
    print('Accumulated return : ', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [ ]:
for i in df.index:
    rtn = df.loc[i, 'return']
    print(rtn)
    break

In [ ]:
returns(df)

In [ ]:
df[~df["trade"].isnull()].head(20)

In [ ]:
df.info()

In [ ]:
def _get_kr_stock_info():
    r_data = []
    params = {
        "ServiceKey" : apisdata["getstockinfo"]["decoding_key"],
        "pageNo" : 1,
        "numOfRows" : 9999,
        "resultType" : "json"
        }
    url = apisdata["getstockinfo"]["request_url"]
    response = requests.get(url, params=params)
    if response.ok:
        get_dict = json.loads(response.text)
        r_data = get_dict["response"]["body"]["items"]["item"]

    r_df = pd.DataFrame(r_data)

    return r_df
    

In [ ]:
df = _get_kr_stock_info()
# 상장폐지(lstgAbolDt) 안되었고, 예탁취소(dpsgCanDt)도 안된 종목만 본다.
f_df = df[(df["lstgAbolDt"] == '') & (df["dpsgCanDt"] == '')]

# 보통주
common_df = f_df[f_df["scrsItmsKcdNm"]=="보통주"]
common_df


In [ ]:
df.head()

In [ ]:
df

In [ ]:
df[df["stckIssuCmpyNm"].str.contains("삼성")]

In [ ]:
kospi_tickers = fdr.StockListing("KOSPI")["Symbol"].tolist()
kospi_tickers

In [ ]:
for ticker in kospi_tickers:
    df = fdr.DataReader(ticker)
    df.head()
    break

In [ ]:
df

In [ ]:
datetime.strptime("09:00:00", "%H:%M:%S").time()

In [ ]:
datetime.combine((datetime.today() + timedelta(days=1)), datetime.strptime("09:00", "%H:%M").time())

In [ ]:
datetime.now().time().hour

In [ ]:
if datetime.now().time() < time(hour=15, minute=9):
    print("aaa")